## Setup

In [ ]:
#@title Setup
import os
# ourddpm_dir = os.path.join("/content", "OurDDPM")

# from getpass import getpass
# print('Make your GitHub Personal access token and insert it below.')
# print('To get GitHub Personal Token,please refer to Section I in Supplementary Material.')
# github_token = getpass('GitHub Personal Token')
# ! git clone https://github.com/Gutianpei/Diffusion-Model-Playground.git $ourddpm_dir

# os.chdir(f'/content/OurDDPM')
os.makedirs("checkpoint", exist_ok=True)
os.makedirs("precomputed", exist_ok=True)
os.makedirs("pretrained", exist_ok=True)
os.makedirs("runs", exist_ok=True)
os.makedirs("runs/interpolation", exist_ok=True)

# ! pip install ftfy regex tqdm


# from utils.colab_utils import GoogleDrive_Dowonloader
# download_with_pydrive = True # param {type:"boolean"}
# dl = GoogleDrive_Dowonloader(download_with_pydrive)


from ourddpm import OurDDPM
from main import dict2namespace
import argparse
import yaml
from PIL import Image
import warnings
warnings.filterwarnings(action='ignore')

device = 'cuda'

%load_ext autoreload
%autoreload 2

In [ ]:
!wget -P pretrained/ https://image-editing-test-12345.s3-us-west-2.amazonaws.com/checkpoints/celeba_hq.ckpt

In [ ]:
import torch
import pdb
import cv2
import glob
# from google.colab.patches import cv2_imshow
import pickle
import matplotlib.pyplot as plt
from math import sqrt

model_path = os.path.join("pretrained/celeba_hq.ckpt")

exp_dir = f"runs/raw_ddpm"
os.makedirs(exp_dir, exist_ok=True)

n_step =  999#@param {type: "integer"}
sampling = "ddpm" #@param ["ddpm", "ddim"]
fixed_xt = True #@param {type: "boolean"}
add_var = True #@param {type: "boolean"}
add_var_on = "0-999" #@param {type: "string"}
vis_gen =  True #@param {type: "boolean"}


args_dic = {
    'config': 'celeba.yml', 
    'n_step': int(n_step), 
    'sample_type': sampling, 
    'eta': 0.0,
    'bs_test': 1, 
    'model_path': model_path, 
    'hybrid_noise': 0, 
    'align_face': 0,
    'image_folder': exp_dir,
    'add_var': bool(add_var),
    'add_var_on': add_var_on
    }
args = dict2namespace(args_dic)

with open(os.path.join('configs', args.config), 'r') as f:
    config_dic = yaml.safe_load(f)
config = dict2namespace(config_dic)


if bool(add_var):
    var_scheduler = []
    periods = add_var_on.split(",")
    for period in periods:
        start = int(period.split("-")[0])
        end = int(period.split("-")[1])
        for n in range(start,end):
            var_scheduler.append(n)

In [ ]:
def normalize(img):
    return (img + 1) / 2

def fuse_and_display(imgs):
    imgs = [normalize(img[0].transpose(1, 2, 0)) for img in imgs]
    display = cv2.hconcat(imgs)
    # display = cv2.cvtColor(display, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20, 40))
    plt.imshow(display)

def display_img(img):
    img = normalize(img[0]).transpose(1, 2, 0)
    plt.figure()
    plt.imshow(img)

## Generate Data

In [ ]:
data_list = []

if bool(fixed_xt):
    torch.manual_seed(0)
    
for i in range(30):
    print(f"Processing {i}th sample...")
    data_dic = {}
    device = torch.device("cuda")
    xt = torch.randn((1,3,256,256),device=device) 
    config.device = device
    runner = OurDDPM(args, config, device=device)

    data_dic["xt"] = xt
    trajs, noise_traj = runner.generate_ddpm_and_noise_traj(xt,var_scheduler)
    data_dic["trajs"] = trajs
    data_dic["noise_traj"] = noise_traj


    # fuse output image
    if bool(vis_gen):
        img_dir = sorted(glob.glob(exp_dir+"/*.png"))[::-1]
        concat = []
        for img in img_dir:
            im = cv2.imread(img)
            concat.append(im)
        concat_img = cv2.hconcat(concat)
        display = concat_img
    else:
        img_dir = sorted(glob.glob(exp_dir+"/*.png"))[0]
        im = cv2.imread(img_dir)
        display = im

    display = cv2.cvtColor(display, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20, 40))
    plt.imshow(display)
    data_list.append(data_dic)
    torch.cuda.empty_cache()
    
    
with open("runs/interpolation/data_1.obj","wb") as f:
    pickle.dump(data_list,f)


## Interpolating $X_T$

In [ ]:
with open("runs/interpolation/data_1.obj","rb") as f:
    data_list = pickle.load(f)


def interpolate_xt(xt1, xt2, steps, noise_traj, var_scheduler):
    
    device = torch.device("cuda")
    config.device = device
    runner = OurDDPM(args, config, device=device)
    res_list = []
    for i in range(steps):
        print(f"Processing {i}th sample...")
        a = i / (steps-1)
        b = (steps-1-i) / (steps-1)
        xt = (a * xt1 + b * xt2) / sqrt(a**2 + b**2)

        traj, noise_traj = runner.generate_ddpm_and_noise_traj(xt, var_scheduler, mode="use", noise_traj=noise_traj)

        res_list.append(traj[-1])
        torch.cuda.empty_cache()
    return res_list

STEPS = 10
results = []

# for i in range(0, 30, 2):
for i in range(16, 17, 2):
    xt1 = data_list[i]["xt"]
    xt2 = data_list[i+1]["xt"]
    res = interpolate_xt(xt1, xt2, STEPS, torch.tensor(data_list[1]["noise_traj"]), var_scheduler)
    results.append(res)

with open("runs/interpolation/data_1_interpolate_xt1.obj","wb") as f:
    pickle.dump(results, f)  


In [ ]:
with open("runs/interpolation/data_1_interpolate_xt1.obj", "rb") as f:
    results = pickle.load(f)
for e in results:
    fuse_and_display(e)


## Interpolate $Z_t$

In [ ]:
with open("runs/interpolation/data_1.obj","rb") as f:
    data_list = pickle.load(f)

def interpolate_zt(zt1, zt2, steps, xt, var_scheduler):
    device = torch.device("cuda")
    config.device = device
    runner = OurDDPM(args, config, device=device)
    res_list = []
    for i in range(steps):
        print(f"Processing {i}th sample...")
        a = i / (steps-1)
        b = (steps-1-i) / (steps-1)
        zt = (a*zt1+b*zt2)/sqrt(a**2+b**2)

        traj, noise_traj = runner.generate_ddpm_and_noise_traj(xt, var_scheduler, mode="use", noise_traj=zt)

        res_list.append(traj[-1])
        torch.cuda.empty_cache()
    return res_list

STEPS = 10
results = []

# for i in range(0, 30, 2):
for i in range(8, 9, 2):
    zt1 = torch.tensor(data_list[i]["noise_traj"])
    zt2 = torch.tensor(data_list[i+1]["noise_traj"])
    res = interpolate_zt(zt1, zt2, STEPS, data_list[i]["xt"], var_scheduler)
    results.append(res)

with open("runs/interpolation/data_1_interpolate_zt2.obj","wb") as f:
    pickle.dump(results, f)

In [ ]:
with open("runs/interpolation/data_1_interpolate_zt2.obj", "rb") as f:
    results = pickle.load(f)
for e in results:
    fuse_and_display(e)


## Interpolate Both

In [ ]:
with open("runs/interpolation/data_1.obj","rb") as f:
    data_list = pickle.load(f)

def interpolate_zt(zt1, zt2, xt1, xt2, steps, var_scheduler):
    device = torch.device("cuda")
    config.device = device
    runner = OurDDPM(args, config, device=device)
    res_list = []
    for i in range(steps):
        print(f"Processing {i}th sample...")
        a = i / (steps-1)
        b = (steps-1-i) / (steps-1)
        zt = (a*zt1+b*zt2) / sqrt(a**2 + b**2)
        xt = (a*xt1+b*xt2) / sqrt(a**2 + b**2)

        traj, noise_traj = runner.generate_ddpm_and_noise_traj(xt, var_scheduler, mode="use", noise_traj=zt)

        res_list.append(traj[-1])
        torch.cuda.empty_cache()
    return res_list

STEPS = 10
results = []

for i in range(0, 30, 2):
# for i in range(0, 2, 2):
    zt1 = torch.tensor(data_list[i]["noise_traj"])
    zt2 = torch.tensor(data_list[i+1]["noise_traj"])
    xt1 = torch.tensor(data_list[i]["xt"])
    xt2 = torch.tensor(data_list[i+1]["xt"])
    res = interpolate_zt(zt1, zt2, xt1, xt2, STEPS, var_scheduler)
    results.append(res)

with open("runs/interpolation/data_1_interpolate_both.obj","wb") as f:
    pickle.dump(results, f)

In [ ]:
with open("runs/interpolation/data_1_interpolate_both.obj", "rb") as f:
    results = pickle.load(f)
for e in results:
    fuse_and_display(e)
